In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from fbprophet import Prophet


%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
plant1_generation = pd.read_csv('../input/solar-power-generation-data/Plant_1_Generation_Data.csv')
plant1_weather = pd.read_csv('../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')
plant2_generation = pd.read_csv('../input/solar-power-generation-data/Plant_2_Generation_Data.csv')
plant2_weather = pd.read_csv('../input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv')

In [ ]:
plant1_generation.head()
# these are midnight hours 00 , yeild is 0 

In [ ]:
plant1_weather.head()
# these are midnight hours 00 , irradiation is 0 

In [ ]:
# Afternoon hours of plant
mask = (plant1_generation['DATE_TIME'] > '15-05-2020 14:00') & (plant1_generation['DATE_TIME'] <= '15-05-2020 14:15')
plant1_generation.loc[mask]

In [ ]:
# Afternoon hours of plant's weather
maskw = (plant1_weather['DATE_TIME'] > '2020-05-15 14:00') & (plant1_weather['DATE_TIME'] <= '2020-05-15 14:15')
plant1_weather.loc[maskw]

In [ ]:
plant2_generation.head()

In [ ]:
plant2_weather.head()

# grid management

predict the power generation for next couple of days 

- Can we identify the need for panel cleaning/maintenance?
- Can we identify faulty or suboptimally performing equipment?

To identify a need for maintenance or fault detection is to build a trend of normal system performance (power output) in a given set of environmental conditions (time of day, weather etc). Any fall in power output indicates a need for maintenance/cleaning or the occurance of a fault. 

# Visualisation

Lets look at the data to see the dialy yeild of plant fluctuating throughout the day

In [ ]:
fig, axes = plt.subplots(nrows = 2, sharex=False, figsize=(20, 5))
fig.subplots_adjust(top = 2.5)
ax = plant1_generation.plot(x = 'DATE_TIME', logy = True, title = 'Plant 1', rot = 20, ax = axes[0]); 
ax = plant2_generation.plot(x = 'DATE_TIME', logy = True, title = 'Plant 2', rot = 20, ax = axes[1]); 

In [ ]:
fig, axes = plt.subplots(nrows = 2, sharex=False, figsize=(20, 5))
fig.subplots_adjust(top = 2.6)
ax = plant1_weather.plot(x = 'DATE_TIME', logy = True, title = 'Plant 1', rot = 20, ax = axes[0]); 
ax = plant2_weather.plot(x = 'DATE_TIME', logy = True, title = 'Plant 2', rot = 20, ax = axes[1]); 

- From the plots above we see distinct peaks in power output, irradiation and temperature, rising and falling with the sun each day
- We can also see in the comparisons between plant 1 and plant 2, that for plant 1 TOTAL_YIELD is consistently > ~10^6 whilst for plant 2 TOTAL_YIELD varies dramatically with several interludes where it falls to zero and at other times in the 10^6 - >10^8 range. 
- Daily weather trends on the other hand appear superficially similar between plant 1 and plant 2

#### Let's compare the distributions of some of these features between the two plants

In [ ]:
power_metrics =  plant1_generation.columns[3:]
weather_metrics = plant1_weather.columns[3:]

In [ ]:
print(power_metrics)
print(weather_metrics)

In [ ]:
# Overlay of TOTAL_YIELD for plant 1 vs plant 2, with IRRADIATION for plant 1 vs plant 2 next to it
# ANOVA for both plant 1 v 2 comparisons below
from scipy import stats
from matplotlib.offsetbox import AnchoredText
from statsmodels.stats.weightstats import ztest

DY_t = stats.ttest_ind(plant1_generation['DAILY_YIELD'],plant2_generation['DAILY_YIELD'])
DY_z = ztest(plant1_generation['DAILY_YIELD'],plant2_generation['DAILY_YIELD'])
# DY_v_1, DY_v_2 = np.var(plant1_generation['DAILY_YIELD']), np.var(plant2_generation['DAILY_YIELD'])
DY_m_1, DY_m_2 = plant1_generation['DAILY_YIELD'].median(), plant2_generation['DAILY_YIELD'].median()

IR_t = stats.ttest_ind(plant1_weather['IRRADIATION'],plant2_weather['IRRADIATION'])
IR_z = ztest(plant1_weather['IRRADIATION'],plant2_weather['IRRADIATION'])
IR_m_1, IR_m_2 = plant1_weather['IRRADIATION'].median(), plant2_weather['IRRADIATION'].median()

AT_t = stats.ttest_ind(plant1_weather['AMBIENT_TEMPERATURE'],plant2_weather['AMBIENT_TEMPERATURE'])
AT_z = ztest(plant1_weather['AMBIENT_TEMPERATURE'],plant2_weather['AMBIENT_TEMPERATURE'])
AT_m_1, AT_m_2 = plant1_weather['AMBIENT_TEMPERATURE'].median(), plant2_weather['AMBIENT_TEMPERATURE'].median()

MT_t = stats.ttest_ind(plant1_weather['MODULE_TEMPERATURE'],plant2_weather['MODULE_TEMPERATURE'])
MT_z = ztest(plant1_weather['MODULE_TEMPERATURE'],plant2_weather['MODULE_TEMPERATURE'])
MT_m_1, MT_m_2 = plant1_weather['MODULE_TEMPERATURE'].median(), plant2_weather['MODULE_TEMPERATURE'].median()

fig, axes = plt.subplots(ncols = 2, nrows = 2, sharex=False, figsize=(20, 5))
fig.subplots_adjust(top = 2)
Alpha = 0.3

anc1 = AnchoredText(f"{DY_t}\n Z test {DY_z}", loc="upper left", frameon=False)
ax1 = sns.distplot(plant1_generation['DAILY_YIELD'], ax=axes[0,0], color = 'b', label = 'Plant 1', hist_kws=dict(alpha=0.2))
ax1 = sns.distplot(plant2_generation['DAILY_YIELD'], ax=axes[0,0], color = 'r', label = 'Plant 2', hist_kws=dict(alpha=0.2))
fig.axes[0].axvline(DY_m_1, color = 'b', alpha=Alpha, label = 'Median Plant 1')
fig.axes[0].axvline(DY_m_2, color = 'r', alpha=Alpha, label = 'Median Plant 2')
ax1.add_artist(anc1)
ax1.legend()

anc2 = AnchoredText(f"{IR_t}\n Z test {IR_z}", loc="upper left", frameon=False)
ax2 = sns.distplot(plant1_weather['IRRADIATION'], ax=axes[0,1], color = 'b', label = 'Plant 1', hist_kws=dict(alpha=0.2))
ax2 = sns.distplot(plant2_weather['IRRADIATION'], ax=axes[0,1], color = 'r', label = 'Plant 2', hist_kws=dict(alpha=0.2))
fig.axes[1].axvline(IR_m_1, color = 'b', alpha=Alpha, label = 'Median Plant 1')
fig.axes[1].axvline(IR_m_2, color = 'r', alpha=Alpha, label = 'Median Plant 2')
ax2.add_artist(anc2)
ax2.legend()

anc3 = AnchoredText(f"{AT_t}\n Z test {AT_z}", loc="upper left", frameon=False)
ax3 = sns.distplot(plant1_weather['AMBIENT_TEMPERATURE'], ax=axes[1,0], color = 'b', label = 'Plant 1', hist_kws=dict(alpha=0.2))
ax3 = sns.distplot(plant2_weather['AMBIENT_TEMPERATURE'], ax=axes[1,0], color = 'r', label = 'Plant 2', hist_kws=dict(alpha=0.2))
fig.axes[2].axvline(AT_m_1, color = 'b', alpha=Alpha, label = 'Median Plant 1')
fig.axes[2].axvline(AT_m_2, color = 'r', alpha=Alpha, label = 'Median Plant 2')
ax3.add_artist(anc3)
ax3.legend()

anc4 = AnchoredText(f"{MT_t}\n Z test {MT_z}", loc="upper left", frameon=False)
ax4 = sns.distplot(plant1_weather['MODULE_TEMPERATURE'], ax=axes[1,1], color = 'b', label = 'Plant 1', hist_kws=dict(alpha=0.2))
ax4 = sns.distplot(plant2_weather['MODULE_TEMPERATURE'], ax=axes[1,1], color = 'r', label = 'Plant 2', hist_kws=dict(alpha=0.2))
fig.axes[3].axvline(MT_m_1, color = 'b', alpha=Alpha, label = 'Median Plant 1')
fig.axes[3].axvline(MT_m_2, color = 'r', alpha=Alpha, label = 'Median Plant 2')
ax4.add_artist(anc4)
ax4.legend();

- From the distributions and Z test results above we can see that although the mean DAILY_YIELD of the two plants is similar; however the median DAILY_YIELD for plant 2 is higher
- The levels of IRRADIATION between the two plants are very similar, though plant 2 has slightly higher irradiation levels > ~0.8
- AMBIENT_TEMPERATURE and MODULE_TEMPERATURE are more variable between plant 1 and plant 2
- Plant 2 has a warmer AMBIENT_TEMPERATURE and MODULE_TEMPERATURE on average. 
- Could these occasional days where ambient temperature rises above 35degC and the higher average module temperature, be the reason for the fluctuations in TOTAL_YIELD seen from plant 2? Perhaps the equipment is over-heating causing it to be less efficient or fail when its preferred operational temperature is exceeded?

# Forecasting Power Output With Facebook Prophet

- Prophet conducts a form of Bayesian time series forecasting, modelling the change in your chosen variable y, with respect to time, using the following summation:

$y(t) = g(t) + s(t) + h(t) + \epsilon_t \tag{1}$

- $ g(t) $ refers to a piece-wise linear/logistic growth model, $ s(t) $ captures seasonality effects, $ h(t) $ models special events (fb calls them holidays) and $\epsilon_t$ models the error
- This type of model is called a generalised additive model and to learn more and build this from scratch in PyMC3, I recommend this link:
https://www.ritchievink.com/blog/2018/10/09/build-facebooks-prophet-in-pymc3-bayesian-time-series-analyis-with-generalized-additive-models/

In [ ]:
df = plant2_generation.copy()  
df.rename(columns= {'DATE_TIME':'ds', 'DAILY_YIELD': 'y'}, inplace = True)
m = Prophet(daily_seasonality=True, weekly_seasonality = True, yearly_seasonality = False)
m.fit(df)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast, xlabel='Date', ylabel='DAILY_YIELD Plant 2',figsize = (15,5))
ax = fig.gca()
ax.set_title("DAILY_YIELD Forecast", size=34)
ax.set_xlabel("X", size=34)
ax.set_ylabel("Y", size=34)
ax.tick_params(axis="x", labelsize=24)
ax.tick_params(axis="y", labelsize=24)

- From the forecast above we can see that the expected trend of DAILY_YIELD for plant 2 is for a slight decline in yield over the next week
- The confidence interval in blue, indicates that the forecast is less certain, the further away from the present we go
- The model has been set up to look for weekly seasonality, which it appears to have captured in its forecast

In [ ]:
fig2 = m.plot_components(forecast, figsize = (11,5))

- In the forecast plots above we can see again the slight downward trend in DAILY_YIELD for the coming week, as well as a tendency for lower DAILY_YIELD on Thursdays seen in the weekly trend
- As expected DAILY_YIELD rises from ~7am onwards with the sun and maxes out by ~9pm each day 

# Detecting anomalous MODULE_TEMPERATURE with Facebook Prophet

- Aside from setting thresholds, whereby if a value is crossed an anomaly is registered, more sophisticated time series anomaly detection can be achieved using models which capture the trend in the data through time and assign confidence intervals, outside of which values are classed as anomalies

- fbprophet can also be used for anomaly detection quite easily
https://facebook.github.io/prophet/docs/outliers.html

- Using the yhat_lower and yhat_upper values (the confidence intervals of the fit model) as thresholds, we can mark any values falling above or below them as "anomalous" values, whilst capturing the normal variation of the time-series

- Many thanks to the authors of https://towardsdatascience.com/anomaly-detection-time-series-4c661f6f165f and https://www.mikulskibartosz.name/prophet-plot-explained/ for the starter code used in this section

In [ ]:
df2 = plant2_weather.copy()  
df2.rename(columns= {'DATE_TIME':'ds', 'MODULE_TEMPERATURE': 'y'}, inplace = True)

In [ ]:
def fit_predict_model(dataframe, interval_width = 0.9, changepoint_range = 0.8):
    m = Prophet(daily_seasonality = True, yearly_seasonality = False, weekly_seasonality = True,
                seasonality_mode = 'multiplicative', 
                interval_width = interval_width,
                changepoint_range = changepoint_range)
    m = m.fit(dataframe)
    
    forecast = m.predict(dataframe)
    forecast['fact'] = dataframe['y'].reset_index(drop = True)
    #fig1 = m.plot(forecast, figsize = (30, 5));
    return forecast
    
pred = fit_predict_model(df2)

In [ ]:
def detect_anomalies(forecast):
    forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
    #forecast['fact'] = df['y']

    forecasted['anomaly'] = 0
    forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
    forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1

    #anomaly importances
    forecasted['importance'] = 0
    forecasted.loc[forecasted['anomaly'] ==1, 'importance'] = \
        (forecasted['fact'] - forecasted['yhat_upper'])/forecast['fact']
    forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] = \
        (forecasted['yhat_lower'] - forecasted['fact'])/forecast['fact']
    
    forecasted['MODULE_TEMPERATURE'] = forecasted['fact']
    return forecasted

pred = detect_anomalies(pred)

In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(ncols = 1, nrows = 1, sharex=False, figsize=(30, 5))
ax = sns.scatterplot(x = pred.index, y = pred['MODULE_TEMPERATURE'], hue = pred['anomaly'], ax=axes, palette = 'bright')
ax = ax.fill_between(pred.index, pred['yhat_lower'], pred['yhat_upper'],color='#0072B2', alpha=0.2)

- An anomaly detection such as this, would allow the operators at the solar plants to identify spikes or drops in MODULE_TEMPERATURE which may be indicative of faults or suboptimal operating conditions and allow them to react accordingly

# Conclusions

- By looking at the distributions we can see clearly that plant 1 and plant 2 are operating in somewhat different environmental conditions. Plant 2 operates in a higher ambient temperature on average, which may put a strain on the equipment, perhap explaining some of it's unusual fluctuations in TOTAL_YIELD
- Using forecasting methods it should definitely be possible to predict likely power outputs for the following week from data such as this. In order to capture larger seasonal trends over the course of a year, one would need a few years worth of training data (ideally though one year might do). fbprophet is a great library for quick and easy time-series forecasting
- Anomaly detection systems can be built from various time series models, provided the models generate confidence intervals which may be use for setting meaningful thresholds. Many approaches exist to capturing the normal trend and variation of the time series and outputting confidence intervals for anomaly detection but the fbprophet one seems to do the job here, given that we have daily/weekly trended data. The sensitivity of any anomaly detection system may be arbitrarily tuned to define "anomalous" data points as being more or less dramatically different from the normal time series trend and this requires some domain knowledge of the system to setup